In [2]:
import numpy as np
import pandas as pd
import os

Загружаем данные, проводим их общий анализ

In [3]:
CURRENT_PATH = './what_do_server_find'

In [4]:
os.listdir('./what_do_server_find')

['toplist.xlsx',
 'datamon.xlsx',
 '.~lock.Описание задач_мониторинг цен.docx#',
 'Описание задач_мониторинг цен.docx',
 '__MACOSX']

In [6]:
# Данный датафрейм содержит наиболее используемые товары
toplist = pd.read_excel('./toplist.xlsx', sheet_name=0)
toplist.head()

,товары
0,Зуб 1085.52.05-1
1,Колесо 3519.05.02.006
2,Вал 3536.11.01.002
3,Вал 3519.05.02.083
4,Ось 3519.21.00.025


In [7]:
# Загрузка основного датасета

In [8]:
datamon = pd.read_excel('./what_do_server_find/datamon.xlsx', sheet_name=0)

In [9]:
# Описание типов данных
datamon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4015 entries, 0 to 4014
Data columns (total 11 columns):
 #   Column                                                  Non-Null Count  Dtype         
---  ------                                                  --------------  -----         
 0   Наименование                                            4015 non-null   object        
 1   Дата поставки                                           4005 non-null   object        
 2   Дата заказа                                             2939 non-null   datetime64[ns]
 3   Срок поставки                                           2939 non-null   float64       
 4   Плановый срок поставки                                  3994 non-null   float64       
 5   Регион                                                  4015 non-null   object        
 6   Объем заказа                                            4015 non-null   int64         
 7   Цена, руб                                               4015

In [10]:
datamon.head()

,Наименование,Дата поставки,Дата заказа,Срок поставки,Плановый срок поставки,Регион,Объем заказа,"Цена, руб",Условия платежа,НРП - нерегламентная потребность (внеплановая закупка),Поставщик
0,ШКИВ К МПЭ-350-900 3536.19.00.200,2017-04-10 00:00:00,2015-06-09,671.0,210.0,Республика Карелия,1,3.839490e+04,ПОСЛЕДУЮЩАЯ ОПЛАТА 100% в течение 30 дней с да...,NaN,4011
1,СТРЕЛА 1085.05.00СБ,2015-11-10 00:00:00,2015-08-03,99.0,180.0,Вологодская область,1,2.050955e+06,ПОСЛЕДУЮЩАЯ ОПЛАТА 100% в течение 60 дней с да...,NaN,3699
2,УСТРОЙСТВО КУЭ-3 04УХЛ2,2015-10-27 00:00:00,2015-08-03,85.0,150.0,Вологодская область,1,1.337580e+06,ПОСЛЕДУЮЩАЯ ОПЛАТА 100% в течение 60 дней с да...,NaN,3699
3,ВТУЛКА 1080.33.70,2015-08-31 00:00:00,2015-08-03,28.0,30.0,Вологодская область,6,3.439490e+03,ПОСЛЕДУЮЩАЯ ОПЛАТА 100% в течение 60 дней с да...,NaN,3699
4,ЗУБ КОВША 1085.52.05-1СБ НАПЛАВКА,2015-08-31 00:00:00,2015-08-03,28.0,75.0,Вологодская область,30,6.496815e+03,ПОСЛЕДУЮЩАЯ ОПЛАТА 100% в течение 60 дней с да...,NaN,3699


Нашей искомой величиной (target) является "Цена, руб", посмотрим с какими другими переменными она коррелирует больше всего, для этого создадим матрицу корреляции

In [11]:
datamon.corr()['Цена, руб']

Срок поставки                                             0.164999
Плановый срок поставки                                    0.133449
Объем заказа                                             -0.020621
Цена, руб                                                 1.000000
НРП - нерегламентная потребность (внеплановая закупка)         NaN
Поставщик                                                -0.189876
Name: Цена, руб, dtype: float64

Наибольшей по модулю корреляцией обладают "Срок поставки" и "Плановый срок поставки", а также "Поставщик", 
однако величина "Срок поставки" определяется уже после закупки, поэтому ключевыми величинами являются 
"Плановый срок поставки" и "Поставщик".

In [13]:
# Проведем анализ, зависит ли цена от месяца и года заказа
# добавим соответствующие переменные

In [14]:
datamon['month'] = datamon['Дата заказа'].dt.month
datamon['year'] = datamon['Дата заказа'].dt.year

In [15]:
datamon.corr()['Цена, руб']

Срок поставки                                             0.164999
Плановый срок поставки                                    0.133449
Объем заказа                                             -0.020621
Цена, руб                                                 1.000000
НРП - нерегламентная потребность (внеплановая закупка)         NaN
Поставщик                                                -0.189876
month                                                     0.050785
year                                                      0.035403
Name: Цена, руб, dtype: float64

In [16]:
# Связь цены с месяцем и годом заказа слабее по отношению к величинам, сказанным выше

In [17]:
# удалим фичи год и месяц
datamon = datamon.drop(['year', 'month'], axis=1)

In [19]:
# Количество уникальных названий
datamon['Наименование'].nunique()

1008

In [20]:
# Количество значений большое, попробуем уменьшить его, создав feature "general_name"
# Многие названия одинаковы, но различаются моделями, поэтому в качестве general_name используем
# первое слово в нижнем регистре из названия
datamon['gen_name'] = datamon['Наименование'].apply(lambda x: x.split()[0].lower())

In [22]:
print('Количество уник. элементов в новой feature:', datamon['gen_name'].nunique())

Количество уник. элементов в новой feature: 168


In [24]:
# Исследуем количество пропусков
datamon.isnull().sum()

Наименование                                                 0
Дата поставки                                               10
Дата заказа                                               1076
Срок поставки                                             1076
Плановый срок поставки                                      21
Регион                                                       0
Объем заказа                                                 0
Цена, руб                                                    0
Условия платежа                                           1818
НРП - нерегламентная потребность (внеплановая закупка)    3712
Поставщик                                                    0
gen_name                                                     0
dtype: int64

In [25]:
# Удалим columns с наибольшим количеством пропусков
datamon = datamon.drop(['НРП - нерегламентная потребность (внеплановая закупка)', 'Условия платежа'], axis=1)

In [27]:
# Также удалим "Срок поставки", а также дату поставки заказа
datamon = datamon.drop(['Срок поставки'], axis=1)

# Создание модели

In [23]:
# Разбиваем данные на X(features), y(target)

In [31]:
# в качестве модели используем алгоритм Xgboost, часто применяемый при решении подобных задач

In [30]:
X = datamon.drop(['Цена, руб'], axis=1)
y = datamon['Цена, руб']

In [32]:
# Предобработка категориальных данных, используем OneHotEncoder
X = pd.get_dummies(X, columns=['Регион'])
X = pd.get_dummies(X, columns=['gen_name'])

In [39]:
X = X.drop(['Наименование', 'Дата поставки', 'Дата заказа'], axis=1)

In [33]:
# Разбиваем данные на train и test
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
# Инициализация модели
from catboost import CatBoostRegressor
cbr = CatBoostRegressor()

In [45]:
# Тренировка модели
cbr.fit(
        X_train, 
        y_train)

Learning rate set to 0.049232
0:	learn: 3663763.2612442	total: 79.6ms	remaining: 1m 19s
1:	learn: 3621845.5387186	total: 80.5ms	remaining: 40.2s
2:	learn: 3579966.3673102	total: 81.4ms	remaining: 27s
3:	learn: 3537099.9328957	total: 82.3ms	remaining: 20.5s
4:	learn: 3497386.9129934	total: 83.4ms	remaining: 16.6s
5:	learn: 3445827.5178872	total: 84.4ms	remaining: 14s
6:	learn: 3407508.2981280	total: 85.3ms	remaining: 12.1s
7:	learn: 3366046.5374655	total: 86.2ms	remaining: 10.7s
8:	learn: 3328532.0222775	total: 87.1ms	remaining: 9.59s
9:	learn: 3293312.3794684	total: 87.9ms	remaining: 8.7s
10:	learn: 3250451.8413275	total: 88.6ms	remaining: 7.97s
11:	learn: 3211536.2609033	total: 89.1ms	remaining: 7.33s
12:	learn: 3179300.1789476	total: 89.9ms	remaining: 6.82s
13:	learn: 3147673.2865909	total: 90.8ms	remaining: 6.4s
14:	learn: 3117527.1580043	total: 91.9ms	remaining: 6.04s
15:	learn: 3085459.6508340	total: 93.5ms	remaining: 5.75s
16:	learn: 3053864.7305045	total: 94.4ms	remaining: 5.46s

260:	learn: 1598030.4759606	total: 334ms	remaining: 947ms
261:	learn: 1597573.9561809	total: 336ms	remaining: 946ms
262:	learn: 1596759.2653403	total: 337ms	remaining: 944ms
263:	learn: 1595251.1264583	total: 338ms	remaining: 941ms
264:	learn: 1593897.4244664	total: 339ms	remaining: 940ms
265:	learn: 1593525.6174636	total: 340ms	remaining: 938ms
266:	learn: 1593168.2426531	total: 341ms	remaining: 935ms
267:	learn: 1591460.3919755	total: 342ms	remaining: 933ms
268:	learn: 1589497.4187478	total: 342ms	remaining: 930ms
269:	learn: 1588213.1102869	total: 343ms	remaining: 928ms
270:	learn: 1587753.8033861	total: 344ms	remaining: 926ms
271:	learn: 1586925.3035682	total: 345ms	remaining: 923ms
272:	learn: 1586136.2631632	total: 350ms	remaining: 931ms
273:	learn: 1585743.6283864	total: 351ms	remaining: 930ms
274:	learn: 1583944.8693214	total: 352ms	remaining: 928ms
275:	learn: 1583111.8613471	total: 353ms	remaining: 926ms
276:	learn: 1582729.8737647	total: 354ms	remaining: 925ms
277:	learn: 15

441:	learn: 1489127.7804078	total: 520ms	remaining: 657ms
442:	learn: 1488854.6807752	total: 521ms	remaining: 655ms
443:	learn: 1488425.0813534	total: 522ms	remaining: 654ms
444:	learn: 1487993.2885440	total: 523ms	remaining: 652ms
445:	learn: 1487653.3402266	total: 524ms	remaining: 651ms
446:	learn: 1487249.8591172	total: 525ms	remaining: 649ms
447:	learn: 1486891.8580493	total: 526ms	remaining: 648ms
448:	learn: 1486521.5611062	total: 527ms	remaining: 646ms
449:	learn: 1486248.1185295	total: 528ms	remaining: 645ms
450:	learn: 1486005.5699578	total: 528ms	remaining: 643ms
451:	learn: 1485732.4014411	total: 529ms	remaining: 642ms
452:	learn: 1485372.7533274	total: 530ms	remaining: 640ms
453:	learn: 1484963.6304692	total: 531ms	remaining: 639ms
454:	learn: 1484762.5975805	total: 532ms	remaining: 638ms
455:	learn: 1484399.8239901	total: 534ms	remaining: 637ms
456:	learn: 1484138.7623154	total: 535ms	remaining: 636ms
457:	learn: 1484017.5951918	total: 536ms	remaining: 634ms
458:	learn: 14

631:	learn: 1447397.8574276	total: 706ms	remaining: 411ms
632:	learn: 1447221.7403579	total: 708ms	remaining: 410ms
633:	learn: 1447125.6172496	total: 709ms	remaining: 409ms
634:	learn: 1447039.8525923	total: 710ms	remaining: 408ms
635:	learn: 1447002.8558367	total: 711ms	remaining: 407ms
636:	learn: 1446881.0191365	total: 711ms	remaining: 405ms
637:	learn: 1446761.3501223	total: 712ms	remaining: 404ms
638:	learn: 1446658.4855967	total: 713ms	remaining: 403ms
639:	learn: 1446458.2601679	total: 714ms	remaining: 402ms
640:	learn: 1446377.0293323	total: 715ms	remaining: 400ms
641:	learn: 1446259.6506551	total: 717ms	remaining: 400ms
642:	learn: 1446153.4319159	total: 718ms	remaining: 399ms
643:	learn: 1446015.3456219	total: 719ms	remaining: 398ms
644:	learn: 1445866.9863459	total: 721ms	remaining: 397ms
645:	learn: 1445716.8771667	total: 722ms	remaining: 395ms
646:	learn: 1445500.4598867	total: 723ms	remaining: 394ms
647:	learn: 1445310.9331303	total: 725ms	remaining: 394ms
648:	learn: 14

825:	learn: 1428441.9286294	total: 893ms	remaining: 188ms
826:	learn: 1428391.1270097	total: 894ms	remaining: 187ms
827:	learn: 1428294.7987085	total: 896ms	remaining: 186ms
828:	learn: 1428223.3788898	total: 897ms	remaining: 185ms
829:	learn: 1428059.5279583	total: 898ms	remaining: 184ms
830:	learn: 1427994.5412653	total: 899ms	remaining: 183ms
831:	learn: 1427945.2892871	total: 900ms	remaining: 182ms
832:	learn: 1427875.9796590	total: 901ms	remaining: 181ms
833:	learn: 1427771.7627582	total: 902ms	remaining: 180ms
834:	learn: 1427683.6488217	total: 903ms	remaining: 178ms
835:	learn: 1427632.5046054	total: 904ms	remaining: 177ms
836:	learn: 1427498.0002060	total: 905ms	remaining: 176ms
837:	learn: 1427388.0408366	total: 906ms	remaining: 175ms
838:	learn: 1427320.7192928	total: 907ms	remaining: 174ms
839:	learn: 1427221.9989933	total: 908ms	remaining: 173ms
840:	learn: 1427177.5812024	total: 909ms	remaining: 172ms
841:	learn: 1427132.0709024	total: 910ms	remaining: 171ms
842:	learn: 14

In [47]:
# В качестве метрики используем RMSE
from sklearn.metrics import mean_squared_error

In [48]:
y_pred = cbr.predict(X_test)

In [50]:
np.sqrt(mean_squared_error(y_test, y_pred))

2759759.537453624

In [51]:
# Другая метрика - MAPE
from sklearn.metrics import mean_absolute_percentage_error

In [52]:
mean_absolute_percentage_error(y_test, y_pred)

36.5625912571718